In [1]:
import pandas as pd
import json
import numpy as np

In [ ]:
df = pd.read_excel('fine_tuning_data.xlsx')
print(df)

In [ ]:
# get the prompt and completion from dataframe, I need the prompt to be a list of strings and the completion to be a list of lists of strings
# I should transfer the data to JSONL
# the format of JSONL is:
# {"prompt": "<prompt text>", "completion": "<ideal generated text>"}
# {"prompt": "<prompt text>", "completion": "<ideal generated text>"}
# {"prompt": "<prompt text>", "completion": "<ideal generated text>"}
prompt = df['prompt'].tolist()
completion = df['completion'].tolist()

print(prompt)
print(completion)

In [ ]:
# process the completion
processed_completion = []
for sent in completion:
    sent = sent.replace(' “completion”: “', '').replace('”}', '')
    processed_completion.append(sent)
print(processed_completion)

In [ ]:
# process the prompt
processed_prompt = []
for line in prompt:
    line = line.replace('{“prompt”: “', '').replace('”', '')
    processed_prompt.append(line)
print(processed_prompt)

In [ ]:
fine_tune_data = []
for prompt, completion in zip(processed_prompt, processed_completion):
    fine_tune_data.append({'prompt': prompt, 'completion': completion})

print(fine_tune_data)

# transfer the fine_tune_data to JSONL and save to train and test file
with open('train.jsonl', 'w', encoding='utf-8') as f:
    for item in train_data:
        json.dump(item, f)
        f.write('\n')
        
with open('test.jsonl', 'w', encoding='utf-8') as f:
    for item in test_data:
        json.dump(item, f)
        f.write('\n')

[{'prompt': '您的快递包裹已经到达，请点击链接进行取件。', 'completion': '诈骗置信度：90%'}, {'prompt': '您的手机卡已经欠费，请点击链接进行充值。', 'completion': '诈骗置信度：90%'}, {'prompt': '您的朋友已经给您转账，请点击链接进行确认。', 'completion': '诈骗置信度：95%'}, {'prompt': '您的车辆违章了，请点击链接进行处理。', 'completion': '诈骗置信度：90%'}, {'prompt': '您的网络账号被投诉了，请点击链接进行申诉。', 'completion': '诈骗置信度：90%'}, {'prompt': '您的电子邮箱被黑客入侵了，请点击链接进行修改密码。', 'completion': '诈骗置信度：90%'}, {'prompt': '您的亲友出了车祸，需要紧急救助，请打款到指定账号。', 'completion': '诈骗置信度：95%'}, {'prompt': '您的银行卡被锁定了，请点击链接进行解锁操作。', 'completion': '诈骗置信度：95%'}, {'prompt': '您的身份证即将过期，需要重新办理，请点击链接进行操作。', 'completion': '诈骗置信度：90%'}, {'prompt': '您的网购订单有问题，需要重新确认，请拨打电话：XXXX。', 'completion': '诈骗置信度：95%'}, {'prompt': '您的电视机需要维修，请点击链接进行预约。', 'completion': '诈骗置信度：90%'}, {'prompt': '您的水电费已经超过限额，需要立刻缴纳，请打款到指定账号。', 'completion': '诈骗置信度：95%'}, {'prompt': '您的房屋合同即将到期，需要重新签订，请点击链接进行操作。', 'completion': '诈骗置信度：90%'}, {'prompt': '您的保险单有问题，需要立刻核实，请拨打电话：XXXX。', 'completion': '诈骗置信度：95%'}, {'prompt': '您的游戏账号被封号了，需要重新激活，请点击链接进行操作。', 'completion': '诈骗置信度：90

In [4]:
import openai

# place your openai api key here
openai.api_key = ''

resp = openai.FineTune.create(training_file='fine_tune_data_prepared_train.jsonl',
                              validation_file='fine_tune_data_prepared_valid.jsonl',
                              model='davinci')

job_id = resp['id']
status = resp['status']
print(f"finetune task id: {job_id}, status: {status}")

InvalidRequestError: No file with ID: fine_tune_data_prepared_train

In [7]:
import openai
job_id= 'ft-THP7R9uCygS0OXDuy5ey6ii6'
status = openai.FineTune.retrieve(id=job_id)["status"]
while status not in ["succeeded", "failed"]:
    time.sleep(2)
    # 获取微调任务的状态
    status = openai.FineTune.retrieve(id=job_id)["status"]
    print(f'fine tune task ID: {job_id}, status: {status}')
    
print(f'finetune task ID: {job_id} finish, finish status: {status}\n')

finetune task ID: ft-THP7R9uCygS0OXDuy5ey6ii6 finish, finish status: succeeded



In [8]:
# assess the model
fine_tune = openai.FineTune.retrieve(id=job_id)
result_files = fine_tune.get("result_files", [])
if result_files:
    result_file = result_files[0]
    resp = openai.File.download(id=result_file["id"])
    print(resp.decode("utf-8"))

step,elapsed_tokens,elapsed_examples,training_loss,training_sequence_accuracy,training_token_accuracy
1,73,1,0.5781850655881862,0.0,0.5882352941176471
2,146,2,0.52123713540951,0.0,0.6470588235294118
3,219,3,0.42551751399539167,0.0,0.7058823529411765
4,308,4,0.2905846079309,0.0,0.6470588235294118
5,381,5,0.2617074795693177,0.0,0.7647058823529411
6,454,6,0.20991373500174149,0.0,0.7647058823529411
7,535,7,0.2506342010646855,0.0,0.8235294117647058
8,632,8,0.045680148717355554,1.0,1.0
9,713,9,0.026783065095028966,1.0,1.0
10,778,10,0.014913069666062172,1.0,1.0
11,859,11,0.061614669514632174,0.0,0.9411764705882353
12,940,12,0.042508664018873876,0.0,0.9411764705882353
13,1021,13,0.06308922533713622,0.0,0.9411764705882353
14,1118,14,0.024790063244526348,0.0,0.9411764705882353
15,1191,15,0.03706049138481034,0.0,0.9411764705882353
16,1280,16,0.029336123288383326,0.0,0.9411764705882353
17,1353,17,0.07983609197335907,0.0,0.9411764705882353
18,1426,18,0.06993664850299976,0.0,0.9411764705882353
19,15

In [10]:
# display the model
model_name = openai.FineTune.retrieve(id=job_id)["fine_tuned_model"]

response = openai.Completion.create(
  model=model_name,
  prompt="今天晚上吃什么好呢？\n",
  temperature=0.7,
  max_tokens=256,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  stop=["END"]
)
print(response['choices'][0]['text'])


小编推荐： >【推荐餐厅】澳门美食：泥鳅叉烧包 >【推荐餐厅】澳门美食：葡式烧肉 >【活动】澳门周末购物绝佳选择 >【活动】澳门餐饮：沙拉酱的制作过程

澳门的晚餐选择还是很多的，

今天小编要说的是经常在澳门见到的餐厅，

据说很不错，在澳
